In [ ]:
import csv

# import DataFrame from pandas in order to convert the sql query in a DataFrame
from pandas import DataFrame

In [ ]:
# The ipython-sql library is loaded using the %load_ext iPython extension
%load_ext sql

In [ ]:
year = '2018'

In [ ]:
%%sql sqlite:///data/prtr_en.db sql_query_result << 

SELECT 
    facilities.name AS LPS, 
    activities.business_sector AS business_sector, 
    activities.prtr_key AS 'PRTR activity',
    nace_code || ': ' || nace_text AS 'NACE',
    activities.prtr_key AS 'GNFR', 
    facilities.administrative_number AS 'E-PRTR/PRTR Facility ID', 
    'TODO' AS 'Height class',
    facilities.wgs84_x AS 'Longitude (deg)', 
    facilities.wgs84_y AS 'Latitude (deg)',

    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Nitrogen oxides (NOx/NO2)') AS "NOx (as NO2) (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Non-methane volatile organic compounds (NMVOC)') 
    AS "NMVOC (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Sulphur oxides (SOx/SO2)') AS "SOx (as SO2) (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Ammonia (NH3)') AS "NH3 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM2.5)') AS "PM2.5 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM10)') AS "PM10 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Carbon monoxide (CO)') AS "CO (kt)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Lead and compounds (as Pb)') AS "Pb (t)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Cadmium and compounds (as Cd)') AS "Cd (t)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Mercury and compounds (as Hg)') AS "Hg (t)",
    SUM(releases.annual_load*1000) FILTER (WHERE substance_name = 'PCDD + PCDF (dioxins + furans)(as Teq)') 
    AS "PCDD/ PCDF (dioxins/ furans)(g I-Teq)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Polycyclic aromatic hydrocarbons (PAHs)') AS "PAHs (t)",
    SUM(releases.annual_load) FILTER (WHERE substance_name = 'Hexachlorobenzene (HCB)') AS "HCB (kg)",
    SUM(releases.annual_load) FILTER (WHERE substance_name = 'Polychlorinated biphenyls') AS "PCBs (kg)"

FROM facilities
    JOIN releases ON facilities.id = releases.facility_ID
    JOIN activities ON releases.facility_ID = activities.facility_ID

WHERE facilities.year = :year AND releases.compartment = 'Air' AND activities.main_activity = 1
GROUP BY facilities.id
ORDER BY name

In [ ]:
data = sql_query_result.DataFrame().fillna("")
data

In [ ]:
# Fill GNFR column with proper values derived from prtr activities and nace codes
gnfr_mapping = {}
with open('data/gnfr_mapping.csv', newline='') as mapping_file: 
    for line in csv.reader(mapping_file, delimiter=';'): 
        gnfr_mapping[line[0]] = line[2]

data.replace({'GNFR': gnfr_mapping}, inplace=True)

# Make sure all point sources have GNFR set
assert len(data[data['GNFR'] == '']) == 0, f"There are still {len(data[data['GNFR'] == ''])} point sources with GNFR listed as '<empty>'!"
assert len(data[~data['GNFR'].str.contains('_')]) == 0, f"There are still {len(data[~data['GNFR'].str.contains('_')])} point sources with GNFR listed as '<prtr key>'!"

In [ ]:
data.to_csv('output/Convert ePRTR data to CLRTAP LPS.csv', sep='|', index=False, columns=['LPS', 'GNFR', 'E-PRTR/PRTR Facility ID', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'NOx (as NO2) (kt)', 'NMVOC (kt)', 'SOx (as SO2) (kt)', 'NH3 (kt)', 'PM2.5 (kt)', 'PM10 (kt)', 'CO (kt)', 'Pb (t)', 'Cd (t)', 'Hg (t)', 'PCDD/ PCDF (dioxins/ furans)(g I-Teq)', 'PAHs (t)', 'HCB (kg)', 'PCBs (kg)'])

In [ ]:
import geopandas

locations = geopandas.GeoDataFrame(data, geometry=geopandas.points_from_xy(data['Longitude (deg)'], data['Latitude (deg)']), crs='epsg:4326')
locations.to_file('output/geo/Convert ePRTR data to CLRTAP LPS.shp', encoding='utf8')

germany = geopandas.read_file('data/geo/germany.geo.json').plot(figsize=(15, 15), alpha=0.3, edgecolor='k')
locations.plot("GNFR", ax=germany, legend=True, figsize=(15, 15), alpha=0.7)